In [3]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import faiss
import time
import os

# Load your dataset
# df = pd.read_csv("test.csv")
products = pd.read_csv('/Users/mdehteshamansari00/Fashion-Recommender-With-AI-Chatbot/Backend/Debug/dataset/products_datafinal_save.csv', on_bad_lines="skip")
new_product = products[['id', 'productDisplayName']]
url=pd.read_csv('/Users/mdehteshamansari00/Fashion-Recommender-With-AI-Chatbot/Backend/Debug/dataset/products_datafinal_save.csv', on_bad_lines="skip")
final=pd.read_csv('/Users/mdehteshamansari00/Fashion-Recommender-With-AI-Chatbot/Backend/Debug/dataset/products_datafinal_save.csv', on_bad_lines="skip")
# Load pre-trained Universal Sentence Encoder
embed = hub.load("/Users/mdehteshamansari00/.cache/kagglehub/models/google/universal-sentence-encoder/tensorFlow2/universal-sentence-encoder/2")


# Check if FAISS index file exists
index_file = "faiss_index.index"
ids_file = "faiss_index_ids.npy"

if os.path.exists(index_file):
    # Load FAISS index
    index = faiss.read_index(index_file)
    # Load the corresponding IDs mapping if exists
    if os.path.exists(ids_file):
        faiss_index_ids = np.load(ids_file)
    else:
        # Extract IDs from new_product
        faiss_index_ids = np.array(new_product['id'])
        # Save the IDs mapping
        np.save(ids_file, faiss_index_ids)
else:
    # Precompute embeddings for all product names
    product_embeddings = embed(new_product['productDisplayName'])

    # Convert embeddings to numpy array
    product_embeddings = np.array(product_embeddings)

    # Convert to float32 (required by FAISS)
    product_embeddings = product_embeddings.astype('float32')

    # Build FAISS index
    index = faiss.IndexFlatIP(product_embeddings.shape[1])
    index.add(product_embeddings)

    # Extract IDs from new_product
    faiss_index_ids = np.array(new_product['id'])
    # Save the IDs mapping
    np.save(ids_file, faiss_index_ids)
    # Save FAISS index
    faiss.write_index(index, index_file)

# User query
user_query = "sun glass"

# Encode user query
query_embedding = embed([user_query])[0].numpy()

# Convert to float32 (required by FAISS)
query_embedding = query_embedding.astype('float32')

# Search for similar products
start_time = time.time()
k = 10  # Number of nearest neighbors to retrieve
distances, indices = index.search(np.expand_dims(query_embedding, axis=0), k)
end_time = time.time()

# Extract the IDs of the most similar products
similar_product_ids = faiss_index_ids[indices[0]]
print(similar_product_ids)
# Print top k most similar products
for i, product_id in enumerate(similar_product_ids):
    print(f"{i+1}. Product ID: {product_id}")

print(f"Search time: {end_time - start_time} seconds")
lst1=[]
for i in similar_product_ids:
    lst=[]
    lst.append(i)
    lst.append(products[products['id']==i]['productDisplayName'].values[0])
    lst.append(url[url['filename']==str(i).strip()+'.jpg']['link'].values[0])
    lst.append(int(products[products['id']==i]['price'].values[0]))
    lst.append(int(products[products['id']==i]['ogprice'].values[0]))
    lst.append(int(products[products['id']==i]['discount'].values[0]))
    lst1.append(lst)
print("\n".join(str(x) for x in lst1))

[114615 179272 178208 177456 180793 179685 178453 177974 178586 177928]
1. Product ID: 114615
2. Product ID: 179272
3. Product ID: 178208
4. Product ID: 177456
5. Product ID: 180793
6. Product ID: 179685
7. Product ID: 178453
8. Product ID: 177974
9. Product ID: 178586
10. Product ID: 177928
Search time: 0.018664121627807617 seconds
[114615, 'Bharatasya Sun Hat', 'https://rukminim1.flixcart.com/image/612/612/xif0q/hat/r/q/s/sun-hat-5-10-years-1-bhht-210423-045-k-sun-hat-bharatasya-original-imagrs4tw8hyvgru.jpeg?q=80', 399, 554, 28]
[179272, 'UPASHNA Glass Kada', 'https://rukminim1.flixcart.com/image/612/612/xif0q/bangle-bracelet-armlet/j/7/q/2-8-2-5-cultured-g-4-haldi-2-8-upashna-original-imagsdc3qvhpvwjh.jpeg?q=80', 251, 303, 17]
[178208, 'PAMSA Glass Bangle', 'https://rukminim1.flixcart.com/image/612/612/xif0q/bangle-bracelet-armlet/x/v/k/2-4-2-4-na-na-24-nb77001-pamsa-original-imagzbc6wsdchupg.jpeg?q=80', 325, 533, 39]
[177456, 'ARCHDECOME Glass Bangle', 'https://rukminim1.flixcart.

In [6]:
import pandas as pd
df = pd.read_csv('/Users/mdehteshamansari00/Fashion-Recommender-With-AI-Chatbot/Backend/Debug/dataset/products_datafinal_save.csv', on_bad_lines="skip")

In [7]:
df

,id,ProductName,ImageUrl,Price,Original_Price,Discount,filename
0,1,"Wrath Colorblock Men Round Neck Black, Grey T-...",https://rukminim1.flixcart.com/image/612/612/k...,264.0,335.28,21.26,1.jpg
1,2,KAEZRI Solid Men Round Neck Black T-Shirt,https://rukminim1.flixcart.com/image/612/612/x...,299.0,343.85,13.04,2.jpg
2,3,"Smartees Printed, Typography Men Round Neck Bl...",https://rukminim1.flixcart.com/image/612/612/x...,219.0,361.35,39.39,3.jpg
3,4,TECHNOSPORT Solid Men Round Neck Black T-Shirt,https://rukminim1.flixcart.com/image/612/612/x...,298.0,467.86,36.31,4.jpg
4,5,NB NICKY BOY Printed Men Round Neck Black T-Shirt,https://rukminim1.flixcart.com/image/612/612/x...,219.0,278.13,21.26,5.jpg
...,...,...,...,...,...,...,...
243217,243218,CAMPUS CAMP BONZAI Running Shoes For Men,https://rukminim1.flixcart.com/image/612/612/x...,1187.0,1412.53,15.97,243218.jpg
243218,243219,CAMPUS CAMP ZANE Running Shoes For Men,https://rukminim1.flixcart.com/image/612/612/x...,1183.0,1857.31,36.31,243219.jpg
243219,243220,CAMPUS CAMP-RELISH Running Shoes For Men,https://rukminim1.flixcart.com/image/612/612/x...,994.0,1828.96,45.65,243220.jpg
243220,243221,WAAN Luxury fashionable black shoes for men an...,https://rukminim1.flixcart.com/image/612/612/x...,799.0,1182.52,32.43,243221.jpg


In [3]:
df['filename'] = df['id'].astype(str) + '.jpg'

In [4]:
df

,id,ProductName,ImageUrl,Price,Original_Price,Discount,filename
0,1,"Wrath Colorblock Men Round Neck Black, Grey T-...",https://rukminim1.flixcart.com/image/612/612/k...,264.0,335.28,21.26,1.jpg
1,2,KAEZRI Solid Men Round Neck Black T-Shirt,https://rukminim1.flixcart.com/image/612/612/x...,299.0,343.85,13.04,2.jpg
2,3,"Smartees Printed, Typography Men Round Neck Bl...",https://rukminim1.flixcart.com/image/612/612/x...,219.0,361.35,39.39,3.jpg
3,4,TECHNOSPORT Solid Men Round Neck Black T-Shirt,https://rukminim1.flixcart.com/image/612/612/x...,298.0,467.86,36.31,4.jpg
4,5,NB NICKY BOY Printed Men Round Neck Black T-Shirt,https://rukminim1.flixcart.com/image/612/612/x...,219.0,278.13,21.26,5.jpg
...,...,...,...,...,...,...,...
243217,243218,CAMPUS CAMP BONZAI Running Shoes For Men,https://rukminim1.flixcart.com/image/612/612/x...,1187.0,1412.53,15.97,243218.jpg
243218,243219,CAMPUS CAMP ZANE Running Shoes For Men,https://rukminim1.flixcart.com/image/612/612/x...,1183.0,1857.31,36.31,243219.jpg
243219,243220,CAMPUS CAMP-RELISH Running Shoes For Men,https://rukminim1.flixcart.com/image/612/612/x...,994.0,1828.96,45.65,243220.jpg
243220,243221,WAAN Luxury fashionable black shoes for men an...,https://rukminim1.flixcart.com/image/612/612/x...,799.0,1182.52,32.43,243221.jpg


In [14]:
df.to_csv('/Users/mdehteshamansari00/Fashion-Recommender-With-AI-Chatbot/Backend/Debug/dataset/products_datafinal_save.csv', index=False)

In [15]:
df.rename(columns={'ProductName':'productDisplayName','ImageUrl':'link','Original_Price':'ogprice','Price':'price','Discount':'discount',}, inplace=True)

In [17]:
df.to_csv('/Users/mdehteshamansari00/Fashion-Recommender-With-AI-Chatbot/Backend/Debug/dataset/products_datafinal_save.csv', index=False)

In [2]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import faiss
import time
import os
# import spacy
# nlp = spacy.load("en_core_web_lg")

# Load your dataset
# df = pd.read_csv("test.csv")
products = pd.read_csv('/Users/mdehteshamansari00/Fashion-Recommender-With-AI-Chatbot/Backend/Debug/dataset/products_datafinal_save.csv', on_bad_lines="skip")
new_product = products[['id', 'productDisplayName']]
#text preprocessing
new_product['productDisplayName'] = new_product['productDisplayName'].str.lower()
#tokenization using spacy
# doc = nlp(new_product['productDisplayName'].values[0])
# tokenization
# new_product['productDisplayName'] = " ".join([token.text for token in doc])

# new_product['productDisplayName'] = new_product['productDisplayName'].str.replace(r'[^\w\s]+', '')
# new_product['productDisplayName'] = new_product['productDisplayName'].str.replace('-','')

# doc = nlp(new_product['productDisplayName'].values[0])
# new_product['productDisplayName'] = " ".join([token.text for token in doc if not token.is_stop and token.is_alpha and not token.is_punct])
url=pd.read_csv('/Users/mdehteshamansari00/Fashion-Recommender-With-AI-Chatbot/Backend/Debug/dataset/products_datafinal_save.csv', on_bad_lines="skip")
final=pd.read_csv('/Users/mdehteshamansari00/Fashion-Recommender-With-AI-Chatbot/Backend/Debug/dataset/products_datafinal_save.csv', on_bad_lines="skip")
# Load pre-trained Universal Sentence Encoder
embed = hub.load("/Users/mdehteshamansari00/.cache/kagglehub/models/google/universal-sentence-encoder/tensorFlow2/universal-sentence-encoder/2")


# Check if FAISS index file exists
index_file = "faiss_index.index"
ids_file = "faiss_index_ids.npy"

if os.path.exists(index_file):
    # Load FAISS index
    index = faiss.read_index(index_file)
    # Load the corresponding IDs mapping if exists
    if os.path.exists(ids_file):
        faiss_index_ids = np.load(ids_file)
    else:
        # Extract IDs from new_product
        faiss_index_ids = np.array(new_product['id'])
        # Save the IDs mapping
        np.save(ids_file, faiss_index_ids)
else:
    # Precompute embeddings for all product names
    product_embeddings = embed(new_product['productDisplayName'])

    # Convert embeddings to numpy array
    product_embeddings = np.array(product_embeddings)

    # Convert to float32 (required by FAISS)
    product_embeddings = product_embeddings.astype('float32')

    # Build FAISS index
    index = faiss.IndexFlatIP(product_embeddings.shape[1])
    index.add(product_embeddings)

    # Extract IDs from new_product
    faiss_index_ids = np.array(new_product['id'])
    # Save the IDs mapping
    np.save(ids_file, faiss_index_ids)
    # Save FAISS index
    faiss.write_index(index, index_file)

# User query
user_query = "sun glass"

# Encode user query
query_embedding = embed([user_query])[0].numpy()

# Convert to float32 (required by FAISS)
query_embedding = query_embedding.astype('float32')

# Search for similar products
start_time = time.time()
k = 10  # Number of nearest neighbors to retrieve
distances, indices = index.search(np.expand_dims(query_embedding, axis=0), k)
end_time = time.time()

# Extract the IDs of the most similar products
similar_product_ids = faiss_index_ids[indices[0]]
print(similar_product_ids)
# Print top k most similar products
for i, product_id in enumerate(similar_product_ids):
    print(f"{i+1}. Product ID: {product_id}")

print(f"Search time: {end_time - start_time} seconds")
lst1=[]
for i in similar_product_ids:
    lst=[]
    lst.append(i)
    lst.append(products[products['id']==i]['productDisplayName'].values[0])
    lst.append(url[url['filename']==str(i).strip()+'.jpg']['link'].values[0])
    lst.append(int(products[products['id']==i]['price'].values[0]))
    lst.append(int(products[products['id']==i]['ogprice'].values[0]))
    lst.append(int(products[products['id']==i]['discount'].values[0]))
    lst1.append(lst)
print("\n".join(str(x) for x in lst1))

/var/folders/lb/0kvsq4m52xz_pn_8rxgbr5lm0000gn/T/ipykernel_59642/2153945967.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_product['productDisplayName'] = new_product['productDisplayName'].str.lower()


[114615 179272 178208 177456 180793 179685 178453 177974 178586 177928]
1. Product ID: 114615
2. Product ID: 179272
3. Product ID: 178208
4. Product ID: 177456
5. Product ID: 180793
6. Product ID: 179685
7. Product ID: 178453
8. Product ID: 177974
9. Product ID: 178586
10. Product ID: 177928
Search time: 0.054186105728149414 seconds
[114615, 'Bharatasya Sun Hat', 'https://rukminim1.flixcart.com/image/612/612/xif0q/hat/r/q/s/sun-hat-5-10-years-1-bhht-210423-045-k-sun-hat-bharatasya-original-imagrs4tw8hyvgru.jpeg?q=80', 399, 554, 28]
[179272, 'UPASHNA Glass Kada', 'https://rukminim1.flixcart.com/image/612/612/xif0q/bangle-bracelet-armlet/j/7/q/2-8-2-5-cultured-g-4-haldi-2-8-upashna-original-imagsdc3qvhpvwjh.jpeg?q=80', 251, 303, 17]
[178208, 'PAMSA Glass Bangle', 'https://rukminim1.flixcart.com/image/612/612/xif0q/bangle-bracelet-armlet/x/v/k/2-4-2-4-na-na-24-nb77001-pamsa-original-imagzbc6wsdchupg.jpeg?q=80', 325, 533, 39]
[177456, 'ARCHDECOME Glass Bangle', 'https://rukminim1.flixcart.